
## Introduction to neural networks and deep learning

## Example - Boston housing dataset



### Dataset description:
- Dataset whose aim is to predic the average price of houses in Boston (in the 1970s) - regression
- Inputs: several features from the house
- 404 training examples; 102 test examples

### Loading and pre-processing the data

In [5]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)
print(max(train_targets), min(train_targets))

(404, 13)
(102, 13)
50.0 5.0


Standardizing data

In [8]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

### Building and training the model

In [13]:
from tensorflow.keras import models, Input
from tensorflow.keras import layers

def build_model(hidden = 64):
    model = models.Sequential()
    model.add(Input((train_data.shape[1],)))
    model.add(layers.Dense(hidden, activation='relu') )
    model.add(layers.Dense(hidden, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [15]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.6660 - mae: 2.5702
20.677095413208008 2.8096306324005127


Example of a cross validation

In [18]:
import numpy as np
k = 5
num_val_samples = len(train_data) // k 
num_epochs = 100
all_scores = []
model = build_model()
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    partial_train_data = np.concatenate( [train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate( [train_targets[:i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print(all_scores)
print(np.mean(all_scores))

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
[1.9743610620498657, 2.3854053020477295, 1.7174699306488037, 1.7081050872802734, 1.480028510093689]
1.8530739784240722
